In [3]:
# Dependencies
import pandas as pd
from numpy import genfromtxt
from google.colab import drive
import os
!pip install pydub
from pydub import AudioSegment

# Create Database

In [4]:
#Load csv file
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#converting all of the mp3 files to .wav because librosa is finicky 

path = '/content/gdrive/My Drive/birdCalls/MP3 files/'

#Change working directory
os.chdir(path)

audio_files = os.listdir()

for file in audio_files:
    #spliting the file into the name and the extension
    name, ext = os.path.splitext(file)
    try:
      if ext == ".mp3":
        mp3_sound = AudioSegment.from_mp3(file)
        #rename them using the old name + ".wav"
        mp3_sound.export("{0}.wav".format(name), format="wav")
    except: 
      print(f"errorrrrr on {file}")

In [0]:
#Now we do a similar iteration in order to remove all old mp3 files

path = '/content/gdrive/My Drive/birdCalls/MP3 files/'
#Change working directory
os.chdir(path)
audio_files = os.listdir()
for file in audio_files:
    #spliting the file into the name and the extension
    name, ext = os.path.splitext(file)
    if ext == ".mp3":
      #delete the mp3 file:
      os.remove(file)

In [0]:
import librosa
import numpy as np

path = '/content/gdrive/My Drive/birdCalls/MP3 files/'

#creating a function that takes in an audio file and outputs a 'scaled mfcc' from librosa
def extract_features(file_name):
   audio, sample_rate = librosa.load(path + file_name, res_type='kaiser_fast') 
   mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
   mfccsscaled = np.mean(mfccs.T,axis=0)
   return mfccsscaled

In [8]:
testing_file = '/content/gdrive/My Drive/birdCalls/testingSet.csv'
testing_file_df = pd.read_csv(testing_file, index_col=0)
testing_file_df.head()

,English Name,id,Recording Name
0,Acorn Woodpecker,475161,475161.mp3
1,Acorn Woodpecker,475160,475160.mp3
2,American Bushtit,545381,545381.mp3
3,American Bushtit,530314,530314.mp3
4,American Bushtit,530313,530313.mp3


In [9]:
#Now we are going to run each recording in testing data CSV through
#the 'extract_features' function defined above 
mfccz = []
for index, row in testing_file_df.iterrows():
    
    file_name = row["Recording Name"]
    name, ext = os.path.splitext(file_name)
    data = extract_features(name + '.wav')
    mfccz.append(data)

# Convert into a Panda dataframe 
testing_file_df["mfccscaled"] = mfccz
testing_file_df.head()

,English Name,id,Recording Name,mfccscaled
0,Acorn Woodpecker,475161,475161.mp3,"[-302.5635967610568, 27.99585196761406, -61.82..."
1,Acorn Woodpecker,475160,475160.mp3,"[-283.1512665095893, 26.62839921722863, -62.28..."
2,American Bushtit,545381,545381.mp3,"[-211.3741573115053, -106.54531916049997, -163..."
3,American Bushtit,530314,530314.mp3,"[-243.45341054857735, -55.88230717297634, -133..."
4,American Bushtit,530313,530313.mp3,"[-335.8557854469277, -58.13038023259462, -134...."


In [10]:
#reformating values in the "Recording Name" column from .mp3 to .wav 
testing_file_df["Recording Name"] = testing_file_df["id"].astype(str) + ".wav"
testing_data = testing_file_df
testing_data.head() 

,English Name,id,Recording Name,mfccscaled
0,Acorn Woodpecker,475161,475161.wav,"[-302.5635967610568, 27.99585196761406, -61.82..."
1,Acorn Woodpecker,475160,475160.wav,"[-283.1512665095893, 26.62839921722863, -62.28..."
2,American Bushtit,545381,545381.wav,"[-211.3741573115053, -106.54531916049997, -163..."
3,American Bushtit,530314,530314.wav,"[-243.45341054857735, -55.88230717297634, -133..."
4,American Bushtit,530313,530313.wav,"[-335.8557854469277, -58.13038023259462, -134...."


In [11]:
#reformating column heading because spaces are weird 

testing_data = testing_data.rename(columns={'English Name': 'english_name'}) 
testing_data.head()

,english_name,id,Recording Name,mfccscaled
0,Acorn Woodpecker,475161,475161.wav,"[-302.5635967610568, 27.99585196761406, -61.82..."
1,Acorn Woodpecker,475160,475160.wav,"[-283.1512665095893, 26.62839921722863, -62.28..."
2,American Bushtit,545381,545381.wav,"[-211.3741573115053, -106.54531916049997, -163..."
3,American Bushtit,530314,530314.wav,"[-243.45341054857735, -55.88230717297634, -133..."
4,American Bushtit,530313,530313.wav,"[-335.8557854469277, -58.13038023259462, -134...."


# Creating a Model


In [0]:
# Dependencies to Visualize the model
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
np.random.seed(0)

In [0]:
# Filepaths and Tensorflow
import tensorflow as tf

In [0]:
# Sklearn scaling and encoding
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [0]:
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist

In [0]:
# Convert features and corresponding classification labels into numpy arrays
X = np.array(testing_data.mfccscaled.tolist())
y = np.array(testing_data.english_name.tolist()) #one hot encoding

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

In [0]:
#splitting the dataaaaa
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [63]:
#setting up a data frame to hold species names and associated index values to make better sense of
#prediction outputs later
y_key = pd.DataFrame(y)
y_key = y_key.drop_duplicates()
y_key = y_key.reset_index()
y_key.head(78)

,index,0
0,0,Acorn Woodpecker
1,2,American Bushtit
2,24,American Cliff Swallow
3,25,American Crow
4,47,American Grey Flycatcher
...,...,...
73,371,Willow Flycatcher
74,374,Wilson's Snipe
75,376,Wood Duck
76,379,Wrentit


In [18]:
#investigating output/ making sure they have compatible dimentsions
print("Training Data Shape:", x_train.shape)
print("Training Data Labels Shape:", y_train.shape)

Training Data Shape: (306, 40)
Training Data Labels Shape: (306, 78)


In [19]:
# the next two code boxes are for exploring input dimensions
x_train[0,:]

array([-2.06596289e+02,  8.57079015e+01, -7.04184014e+01, -5.43944642e+00,
       -3.88994931e+01,  8.85734932e+00, -1.74121000e+01,  7.00287227e+00,
       -1.97725823e+01,  6.36273336e+00, -1.38822080e+01,  8.67868399e+00,
       -6.51724486e+00,  6.78793949e+00, -8.51225016e+00,  4.97283930e+00,
       -9.87562775e+00,  2.25611433e+00, -8.96081715e+00,  3.61874199e+00,
       -4.51923053e+00,  1.21292485e+00, -2.67505005e+00,  2.22881352e+00,
       -3.00849762e+00,  3.60755093e+00, -7.76440928e-01,  2.20827721e+00,
        1.90741826e-01,  1.89549428e-02, -2.90814121e+00, -3.17419083e-02,
       -3.89405778e+00, -2.27997080e-01, -1.88911463e+00,  2.55132468e-02,
       -4.99669860e-01, -3.98460918e-01, -1.98136431e+00,  6.05728997e-01])

In [20]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [0]:
# Create an empty sequential model
model = Sequential()

In [0]:
# Add the first hidden layer
# Number of nodes is 780 which is ten times the ammount of categories (distict bird species)
# `relu` is a common activation function
# we must also specify the dimension of our input layer. input_dim=x_train.shape[1] = 40 (40 values in each mfcc matrix)).
model.add(Dense(780, activation='relu', input_dim=x_train.shape[1]))

In [0]:
#second hidden layer
#A dense layer is when every node from the previous layer is connected to each node in the current layer.
model.add(Dense(780, activation='relu'))

In [0]:
#We also need to specify the number of output classes. 
#In this case, the number of species that we wish to classify
model.add(Dense(78, activation='softmax'))

In [0]:
# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 780)               31980     
_________________________________________________________________
dense_1 (Dense)              (None, 780)               609180    
_________________________________________________________________
dense_2 (Dense)              (None, 520)               406120    
_________________________________________________________________
dense_3 (Dense)              (None, 780)               406380    
_________________________________________________________________
dense_4 (Dense)              (None, 78)                60918     
Total params: 1,514,578
Trainable params: 1,514,578
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Output layer in this example is using software for logistic regression (categorical)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [30]:
#Training consists of updating our weights using our optimizer and loss function.
#In this example, we choose 100 iterations (loops) of training that are called epochs.

model.fit(
    x_train,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
10/10 - 0s - loss: 12.9090 - accuracy: 0.0294
Epoch 2/100
10/10 - 0s - loss: 4.7016 - accuracy: 0.0817
Epoch 3/100
10/10 - 0s - loss: 3.8773 - accuracy: 0.1405
Epoch 4/100
10/10 - 0s - loss: 3.5659 - accuracy: 0.1373
Epoch 5/100
10/10 - 0s - loss: 3.3200 - accuracy: 0.2255
Epoch 6/100
10/10 - 0s - loss: 3.1256 - accuracy: 0.2026
Epoch 7/100
10/10 - 0s - loss: 2.9481 - accuracy: 0.2026
Epoch 8/100
10/10 - 0s - loss: 2.7094 - accuracy: 0.2712
Epoch 9/100
10/10 - 0s - loss: 2.5331 - accuracy: 0.2908
Epoch 10/100
10/10 - 0s - loss: 2.3160 - accuracy: 0.3039
Epoch 11/100
10/10 - 0s - loss: 2.1911 - accuracy: 0.3431
Epoch 12/100
10/10 - 0s - loss: 2.0928 - accuracy: 0.3562
Epoch 13/100
10/10 - 0s - loss: 1.8114 - accuracy: 0.4150
Epoch 14/100
10/10 - 0s - loss: 1.6690 - accuracy: 0.4739
Epoch 15/100
10/10 - 0s - loss: 1.6791 - accuracy: 0.4739
Epoch 16/100
10/10 - 0s - loss: 1.4932 - accuracy: 0.5131
Epoch 17/100
10/10 - 0s - loss: 1.4898 - accuracy: 0.5294
Epoch 18/100
10/10 - 0

In [0]:
#saving model
model.save("/content/gdrive/My Drive/birdCalls/bird_model_trained.h5")

# Evaluating our Model

In [0]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model("/content/gdrive/My Drive/birdCalls/bird_model_trained.h5")

In [72]:
# Evaluate the model using the testing data 
model_loss, model_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3/3 - 0s - loss: 7.3277 - accuracy: 0.4026
Loss: 7.32773494720459, Accuracy: 0.4025973975658417


In [73]:
# Evaluate the model using the training data 
model_loss, model_accuracy = model.evaluate(x_train, y_train, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 0.1384 - accuracy: 0.9575
Loss: 0.13835513591766357, Accuracy: 0.9575163125991821


# Making Predictions

In [74]:
# Grab just one data point to test with
test = np.expand_dims(x_train[0], axis=0)
test

array([[-2.06596289e+02,  8.57079015e+01, -7.04184014e+01,
        -5.43944642e+00, -3.88994931e+01,  8.85734932e+00,
        -1.74121000e+01,  7.00287227e+00, -1.97725823e+01,
         6.36273336e+00, -1.38822080e+01,  8.67868399e+00,
        -6.51724486e+00,  6.78793949e+00, -8.51225016e+00,
         4.97283930e+00, -9.87562775e+00,  2.25611433e+00,
        -8.96081715e+00,  3.61874199e+00, -4.51923053e+00,
         1.21292485e+00, -2.67505005e+00,  2.22881352e+00,
        -3.00849762e+00,  3.60755093e+00, -7.76440928e-01,
         2.20827721e+00,  1.90741826e-01,  1.89549428e-02,
        -2.90814121e+00, -3.17419083e-02, -3.89405778e+00,
        -2.27997080e-01, -1.88911463e+00,  2.55132468e-02,
        -4.99669860e-01, -3.98460918e-01, -1.98136431e+00,
         6.05728997e-01]])

In [37]:
# Make a prediction. The result should be 0000010000000 for a 5
model.predict(test).round()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [80]:
print(f"One-Hot-Encoded Prediction: {model.predict(test).round()}")
print(f"Predicted class: {model.predict_classes(test)}")
bingo = model.predict_classes(test)
print(y_key.iloc[bingo])


One-Hot-Encoded Prediction: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]]
Predicted class: [24]
    index                     0
24    163  California Scrub Jay


In [76]:
# Grab just one data point to test with (THIS TIME FROM x_test instead of x_train)
test2 = np.expand_dims(x_test[0], axis=0)
test2

array([[-253.08723963,  -91.06396757, -161.60761147,  -11.33920562,
         -34.7998268 ,   12.21030263,  -10.74922761,   16.04121826,
          -6.32872337,   20.85129388,   -4.06108139,   11.77772012,
          -7.19477238,   11.20803265,   -8.31571606,    5.1707778 ,
         -10.09654712,    4.69632146,   -7.57887874,    4.17742128,
          -3.9277244 ,    2.69688135,   -2.09687446,    3.36847486,
          -2.22993167,    2.91704796,   -1.56591919,    2.23664619,
          -1.91618888,    2.27588791,   -2.75245022,    1.7501884 ,
          -1.62542113,    0.87500893,   -1.77393805,    1.77158875,
          -0.35764993,    0.68178928,   -0.91397937,    1.0106143 ]])

In [77]:
model.predict(test2).round()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

In [82]:
print(f"One-Hot-Encoded Prediction: {model.predict(test2).round()}")
print(f"Predicted class: {model.predict_classes(test2)}")
bingo2 = model.predict_classes(test2)
print(y_key.iloc[bingo2])

One-Hot-Encoded Prediction: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0.]]
Predicted class: [70]
    index                        0
70    353  White-breasted Nuthatch


In [83]:
#We see below that our prediction was incorrect. The one is not in the 70th index 
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)